<a href="https://colab.research.google.com/github/GaworO/Sales/blob/main/Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This challenge serves as final project for the "How to win a data science competition" Coursera course.
#In this competition you will work with a challenging time-series dataset consisting of daily sales data, 
#kindly provided by one of the largest Russian software firms - 1C Company. 
#We are asking you to predict total sales for every product and store in the next month. 
#By solving this competition you will be able to apply and enhance your data science skills.

In [ ]:
#ID - an Id that represents a (Shop, Item) tuple within the test set
#shop_id - unique identifier of a shop
#item_id - unique identifier of a product
#item_category_id - unique identifier of item category
#item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
#item_price - current price of an item
#date - date in format dd/mm/yyyy
#date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
#item_name - name of item
#shop_name - name of shop
#item_category_name - name of item category
#This dataset is permitted to be used for any purpose, including commercial use.

In [ ]:
import pandas as pd
#from googletrans import Translator
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
item_categories = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle Sales/data/item_categories.csv", sep=",")
items = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle Sales/data/items.csv", sep=",")
sales_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle Sales/data/sales_train.csv", sep=",")
sample_submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle Sales/data/sample_submission.csv", sep=",")
shops = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle Sales/data/shops.csv", sep=",")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Kaggle Sales/data/test.csv", sep=",")

In [ ]:
#The task is to forecast the total amount of products sold in every shop for the test set.

In [ ]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [ ]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [ ]:
item_categories = item_categories.join(items.set_index('item_category_id'), on='item_category_id')

In [ ]:
#select one category id as an exmple
item_categories[item_categories["item_category_id"] == 51]

,item_category_name,item_category_id,item_name,item_id
51,Книги - Познавательная литература,51,Правила дорожного движения 2009. Новейшая мето...,17703


In [ ]:
#join item categories with sales prodcuts to get the date and prices
sales_products = sales_train.join(item_categories.set_index("item_id"), on="item_id")

In [ ]:
sales_products.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_name,item_category_id,item_name
0,02.01.2013,0,59,22154,999.00,1.0,Кино - Blu-Ray,37,ЯВЛЕНИЕ 2012 (BD)
1,03.01.2013,0,25,2552,899.00,1.0,Музыка - Винил,58,DEEP PURPLE The House Of Blue Light LP
2,05.01.2013,0,25,2552,899.00,-1.0,Музыка - Винил,58,DEEP PURPLE The House Of Blue Light LP
3,06.01.2013,0,25,2554,1709.05,1.0,Музыка - Винил,58,DEEP PURPLE Who Do You Think We Are LP
4,15.01.2013,0,25,2555,1099.00,1.0,Музыка - CD фирменного производства,56,DEEP PURPLE 30 Very Best Of 2CD (Фирм.)


In [ ]:
sales_products[sales_products["shop_id"] == 59].loc[sales_products['item_id'] == 22151]
#.where("sales_products" == "22154")

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_name,item_category_id,item_name
40084,10.01.2013,0,59,22151,399.0,1.0,Кино - DVD,40,ЯВЛЕНИЕ (2012)
78663,02.01.2013,0,59,22151,399.0,1.0,Кино - DVD,40,ЯВЛЕНИЕ (2012)
342905,30.03.2013,2,59,22151,399.0,1.0,Кино - DVD,40,ЯВЛЕНИЕ (2012)


In [ ]:
#have a look which categories have the most and the least products
item_categories.groupby(["item_category_id"]).agg({"item_id": "count"}).sort_values(by="item_id", ascending=False)

,item_id
item_category_id,
40,5035
55,2365
37,1780
31,1125
58,790
...,...
1,2
50,2
10,1


In [ ]:
sales_products[sales_products["shop_id"] == 59]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_name,item_category_id,item_name
0,02.01.2013,0,59,22154,999.0,1.0,Кино - Blu-Ray,37,ЯВЛЕНИЕ 2012 (BD)
40084,10.01.2013,0,59,22151,399.0,1.0,Кино - DVD,40,ЯВЛЕНИЕ (2012)
77502,04.01.2013,0,59,5603,699.0,1.0,Аксессуары - PSVita,5,PS Vita: Чехол 4gamers Travel Case дорожный го...
77503,19.01.2013,0,59,5587,199.0,2.0,Аксессуары - PSVita,5,PS Vita/PSP: Наушники Hama HK-3023
77504,31.01.2013,0,59,5613,5571.0,1.0,Аксессуары - PS3,2,PS3: Гарнитура беспроводная для PS3 (с поддерж...
...,...,...,...,...,...,...,...,...,...
2899654,10.10.2015,33,59,4962,298.0,1.0,Игры PC - Стандартные издания,30,"Murdered: Soul Suspect [PC, Jewel, русская вер..."
2899655,04.10.2015,33,59,4706,299.0,1.0,Музыка - CD локального производства,55,MUSE Drones
2899656,11.10.2015,33,59,5002,2999.0,1.0,Игры - PS3,19,"NBA 2K16 [PS3, русская документация]"
2899657,15.10.2015,33,59,5233,1199.0,1.0,Игры - PS3,19,"Need for Speed Rivals (Essentials) [PS3, русск..."
